# Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

#### Re-run Part I of the assignment and obtain the df

In [120]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import requests
#Import table from URL and create PANDAS
rst = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
table = pd.read_html(rst.text)
df=pd.DataFrame(table[0])
#Assign columns: PostalCode, Borough, and Neighborhood
df.columns=['Postcode','Borough','Neighbourhood']
df.drop([0],axis=0,inplace=True)
df.reset_index()
# Remove rows with "Not assigned" in "Borough" column
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head()
#merge rows with same Postcode
df1=df.groupby("Postcode").agg(lambda x:','.join(set(x)))
df1.head()
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df1.loc[df1['Neighbourhood']=="Not assigned",'Neighbourhood']=df1.loc[df1['Neighbourhood']=="Not assigned",'Borough']
df1.shape

(103, 2)

#### Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

##### Re-run part II - create new df with coordinates

In [121]:
#import csv in df
df_csv = pd.read_csv('http://cocl.us/Geospatial_data')
#change the column name
df_csv.columns=['Postcode', 'Latitude', 'Longitude']
#merge the 2 df
df_merge = pd.merge(df, df_csv)
df_merge=df_merge.groupby("Postcode").agg({
                                        'Borough':'first',
                                        'Latitude':'first',
                                        'Longitude':'first',
                                        'Neighbourhood':','.join})
df_merge.head()

,Borough,Latitude,Longitude,Neighbourhood
Postcode,,,,
M1B,Scarborough,43.806686,-79.194353,"Rouge,Malvern"
M1C,Scarborough,43.784535,-79.160497,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,43.763573,-79.188711,"Guildwood,Morningside,West Hill"
M1G,Scarborough,43.770992,-79.216917,Woburn
M1H,Scarborough,43.773136,-79.239476,Cedarbrae


#### Part III: Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data.

In [122]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Create a table of top 20 venues per neighbourhood "Downtown Toronto"


In [123]:
#Create a map of Toronto with neighborhoods superimposed on top.¶
T_latitude = 43.6532 
T_longitude = -79.3832
map = folium.Map(location = [T_latitude, T_longitude], zoom_start = 12)
# add markers to map
for lat, lng, borough, neighborhood in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Borough'], df_merge['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='YlOrRd',
        fill_opacity=0.7).add_to(map) 
map

In [124]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'ZLLFAV02V3JLIRJADQ4ZEPJJX5TS4OQQCR0Z2ESI3RFNFHSH' # your Foursquare ID
CLIENT_SECRET = 'AD3GHTHRR43V424PMWXOLGU5TRDTO5MWXAC3ZMCP1OVED5UC' # your Foursquare Secret
VERSION = '201904025' # Foursquare API version

#slice the original dataframe and create a new dataframe of the Etobicoke data
Etobicoke_data = df_merge[df_merge['Borough'] == 'Etobicoke'].reset_index(drop=True)
Etobicoke_data

,Borough,Latitude,Longitude,Neighbourhood
0,Etobicoke,43.605647,-79.501321,"Humber Bay Shores,Mimico South,New Toronto"
1,Etobicoke,43.602414,-79.543484,"Alderwood,Long Branch"
2,Etobicoke,43.653654,-79.506944,"The Kingsway,Montgomery Road,Old Mill North"
3,Etobicoke,43.636258,-79.498509,"Humber Bay,King's Mill Park,Kingsway Park Sout..."
4,Etobicoke,43.628841,-79.520999,"Kingsway Park South West,Mimico NW,The Queensw..."
5,Etobicoke,43.667856,-79.532242,Islington Avenue
6,Etobicoke,43.650943,-79.554724,"Cloverdale,Islington,Martin Grove,Princess Gar..."
7,Etobicoke,43.643515,-79.577201,"Bloordale Gardens,Eringate,Markland Wood,Old B..."
8,Etobicoke,43.696319,-79.532242,Westmount
9,Etobicoke,43.688905,-79.554724,"Kingsview Village,Martin Grove Gardens,Richvie..."


In [125]:
#get the geographical coordinates of Etobicoke.
address = 'Etobicoke, Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

#Create a map of Etobicoke
TE_latitude = 43.67145915 
TE_longitude = -79.5524920661167
mapTE = folium.Map(location = [TE_latitude, TE_longitude], zoom_start = 12)

mapTE

The geograpical coordinate of Etobicoke are 43.67145915, -79.5524920661167.


### Explore the "Etobicoke" neighborhood in our dataframe.

In [126]:
#explore the "Etobicoke" neighborhood in our dataframe.
#Get the neighborhood's name.
Etobicoke_data.loc[0, 'Neighbourhood']

ng_latitude = Etobicoke_data.loc[0, 'Latitude'] # neighborhood latitude value
ng_longitude = Etobicoke_data.loc[0, 'Longitude'] # neighborhood longitude value

ng_name = Etobicoke_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(ng_name, 
                                                               ng_latitude, 
                                                               ng_longitude))



Latitude and longitude values of Humber Bay Shores,Mimico South,New Toronto are 43.6056466, -79.50132070000001.


In [127]:
#get the top 10 venues that are in Etobicoke within a radius of 500 meters
LIMIT = 10 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    ng_latitude, 
    ng_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ZLLFAV02V3JLIRJADQ4ZEPJJX5TS4OQQCR0Z2ESI3RFNFHSH&client_secret=AD3GHTHRR43V424PMWXOLGU5TRDTO5MWXAC3ZMCP1OVED5UC&v=201904025&ll=43.6056466,-79.50132070000001&radius=500&limit=10'

In [128]:
#Send the GET request and examine the resutls
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ca69f95db04f507f744271c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 14,
  'suggestedBounds': {'ne': {'lat': 43.6101466045, 'lng': -79.49511771930959},
   'sw': {'lat': 43.6011465955, 'lng': -79.50752368069043}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b119977f964a520488023e3',
       'name': 'LCBO',
       'location': {'address': '2762 Lake Shore Blvd W',
        'crossStreet': 'btwn 1st & 2nd St',
        'lat': 43.60228082768786,
        'lng': -79.4993016827402,
        'labeledLatLngs': [{'label': 'display',
          'lat':

In [129]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
#clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,LCBO,Liquor Store,43.602281,-79.499302
1,New Toronto Fish & Chips,Restaurant,43.601849,-79.503281
2,Domino's Pizza,Pizza Place,43.601676,-79.500908
3,Delicia Bakery & Pastry,Bakery,43.601403,-79.503012
4,Lucky Dice Restaurant,Café,43.601392,-79.503056
5,Subway,Sandwich Place,43.602382,-79.498275
6,McDonald's,Fast Food Restaurant,43.602470,-79.498963
7,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.602069,-79.499400
8,Maple Leaf House,American Restaurant,43.602040,-79.498678
9,Shoppers Drug Mart,Pharmacy,43.601611,-79.502164


In [130]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


In [131]:
#create a function to repeat the same process to all the neighborhoods in Etobicoke
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
# create a new dataframe 
Etobicoke_venues = getNearbyVenues(names=Etobicoke_data['Neighbourhood'],
                                   latitudes=Etobicoke_data['Latitude'],
                                   longitudes=Etobicoke_data['Longitude']
                                  )
print(Etobicoke_venues.shape)
Etobicoke_venues.head()


Humber Bay Shores,Mimico South,New Toronto
Alderwood,Long Branch
The Kingsway,Montgomery Road,Old Mill North
Humber Bay,King's Mill Park,Kingsway Park South East,Mimico NE,Old Mill South,The Queensway East,Royal York South East,Sunnylea
Kingsway Park South West,Mimico NW,The Queensway West,Royal York South West,South of Bloor
Islington Avenue
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Westmount
Kingsview Village,Martin Grove Gardens,Richview Gardens,St. Phillips
Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown
Northwest
(63, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321,LCBO,43.602281,-79.499302,Liquor Store
1,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321,New Toronto Fish & Chips,43.601849,-79.503281,Restaurant
2,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321,Domino's Pizza,43.601676,-79.500908,Pizza Place
3,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321,Delicia Bakery & Pastry,43.601403,-79.503012,Bakery
4,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321,Lucky Dice Restaurant,43.601392,-79.503056,Café


In [132]:
#Analyze Each Neighborhood
Etobicoke_onehot = pd.get_dummies(Etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")
Etobicoke_onehot['Neighborhood'] = Etobicoke_venues['Neighborhood'] 
fixed_columns = [Etobicoke_onehot.columns[-1]] + list(Etobicoke_onehot.columns[:-1])
Etobicoke_onehot = Etobicoke_onehot[fixed_columns]
Etobicoke_onehot.head()

,Neighborhood,American Restaurant,Bakery,Bank,Bar,Baseball Field,Beer Store,Breakfast Spot,Burger Joint,Bus Line,...,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Skating Rink,Smoke Shop,Supplement Shop,Tanning Salon,Wings Joint
0,"Humber Bay Shores,Mimico South,New Toronto",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Humber Bay Shores,Mimico South,New Toronto",0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,"Humber Bay Shores,Mimico South,New Toronto",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Humber Bay Shores,Mimico South,New Toronto",0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Humber Bay Shores,Mimico South,New Toronto",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [133]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
Etobicoke_grouped = Etobicoke_onehot.groupby('Neighborhood').mean().reset_index()
#print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in Etobicoke_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Etobicoke_grouped[Etobicoke_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
#put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
#create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0        Grocery Store  0.22
1             Pharmacy  0.11
2          Pizza Place  0.11
3           Beer Store  0.11
4  Fried Chicken Joint  0.11


----Alderwood,Long Branch----
          venue  freq
0   Pizza Place  0.22
1           Gym  0.11
2   Coffee Shop  0.11
3      Pharmacy  0.11
4  Skating Rink  0.11


----Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe----
               venue  freq
0  Convenience Store  0.17
1           Pharmacy  0.17
2        Pizza Place  0.17
3         Beer Store  0.17
4               Café  0.17


----Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park----
                 venue  freq
0                 Bank   1.0
1  American Restaurant   0.0
2  Rental Car Location   0.0
3    Mobile Phone Shop   0.0
4                 Park   0.0


----Humber Bay Shores,Mimico South,New Toronto----
                

NameError: name 'manhattan_grouped' is not defined

### Cluster Neighborhoods

In [147]:
# set number of clusters
kclusters = 5
Etobicoke_grouped_clustering = Etobicoke_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Etobicoke_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Etobicoke_merged = Etobicoke_data
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Etobicoke_merged = Etobicoke_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
Etobicoke_merged.head() # check the last columns!
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

ValueError: cannot insert Cluster Labels, already exists

In [ ]:
# add markers to map
for lat, lng, borough, neighborhood in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Borough'], df_merge['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='YlOrRd',
        fill_opacity=0.7).add_to(map)  
map

In [ ]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
T_latitude = 43.6532 
T_longitude = -79.3832
map = folium.Map(location = [T_latitude, T_longitude], zoom_start = 12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Borough'], df_merge['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='YlOrRd',
        fill_opacity=0.7).add_to(map)  
    

map